In [2]:
import pandas as pd
user = pd.read_csv("users.csv")
item = pd.read_csv("user_item.csv",sep="\t")


In [3]:
item.shape

(6319946, 4)

In [4]:
item[item.user_id.isin(user_target.user_id)].shape

(2456196, 4)

In [5]:
item[~item.user_id.isin(user_target.user_id)].user_id.nunique()

690442

In [6]:
item_ids = item.item_id.unique()
user_ids = item.user_id.unique()
item2idx = {item:idx for idx,item in enumerate(item_ids)}
idx2item = {idx:item for item,idx in item2idx.items()}
user2idx = {user:idx for idx,user in enumerate(user_ids)}
idx2user = {idx:user for idx,user in enumerate(user_ids)}

In [7]:
item.item_id.max(),len(item.item_id.unique()),len(user2idx),len(item2idx)

('I99999', 328823, 913707, 328823)

In [8]:
max([int(x[1:]) for x in item.item_id.values]), len(item2idx)

(355712, 328823)

### user feature

In [9]:
user_master = pd.read_csv("user_master.tsv",sep="\t")

In [10]:
user_master.shape,user_master.user_id.max(),user_master.user_id.nunique()

((1640956, 16), 1640956, 1640956)

In [11]:
user_feature = user_master[user_master.user_id.isin(set(user_ids).union(set(user_target.user_id)))]
user_feature = user_feature.reset_index(drop=True)
user_feature.shape

(940785, 16)

In [12]:
# user_feature = user_feature[["user_id"]]
user_feature.shape

(940785, 16)

In [45]:
len(set(user_target.user_id)-set(item.user_id))

27078

In [13]:
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature1"])],axis=1)
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature2"])],axis=1)
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature3"])],axis=1)
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature4"])],axis=1)
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature5"])],axis=1)
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature6"])],axis=1)

In [17]:
user_feature.columns

Index(['user_id', 'feature1', 'feature2', 'feature3', 'feature4', 'feature5',
       'feature6', 'feature_list1', 'feature_list2',
       'feature_list2_frequencies',
       ...
       'F1', 'F10', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
      dtype='object', length=102)

In [18]:
user_feature["A1"].sum(),user_feature["A2"].sum(),user_feature["A3"].sum()

(691693, 249091, 1)

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
def multihot_encode(user_fea, target, topk = 9999):
    feature_list = user_fea[target].apply(lambda x:str(x).split(',')).values
    feature_list_multihot = MultiLabelBinarizer().fit_transform(feature_list)
    common_feature_idx = np.argsort(feature_list_multihot.sum(axis=0))[-topk:]
    common_feature_idx = common_feature_idx[::-1]
    # print(feature_list_multihot.sum(axis=0)[common_feature_idx])
    # feature_list_multihot = feature_list_multihot.astype(np.bool_)
    extra_feature = pd.DataFrame(feature_list_multihot[:,common_feature_idx],dtype=bool)
    extra_feature.columns=[target+"_"+str(ii) for ii in range(extra_feature.shape[1])]
    # print(extra_feature.memory_usage())
    return extra_feature

In [ ]:
parsed_list3 = user_feature["feature_list3"].apply(lambda x:str(x).split(',')).values
parsed_list2 = user_feature["feature_list2"].apply(lambda x:str(x).split(',')).values
parsed_list4 = user_feature["feature_list4"].apply(lambda x:str(x).split(',')).values
parsed_list5 = user_feature["feature_list5"].apply(lambda x:str(x).split(',')).values

In [ ]:
list_fea = [y for x in parsed_list2 for y in x]
len(set(list_fea))

66837

In [15]:
for i in [1,3,5]:
    feature_list_multihot = multihot_encode(user_feature,f"feature_list{i}",10)
    user_feature = pd.concat([user_feature,feature_list_multihot],axis=1)

In [21]:
# user_feature

In [ ]:
list(user_feature.columns)

In [16]:
import importlib
import BPR_lib
importlib.reload(BPR_lib)
from BPR_lib import reduce_mem
user_feature = reduce_mem(user_feature)

-- Mem. usage decreased to 215.33 Mb (1.6% reduction),time spend:0.00 min


### hist / last

In [17]:
item = item.sort_values(by=['user_id', 'latest_timestamp'])
item_last = item.groupby("user_id").tail(1)
def hist_func(user_df):
    if len(user_df) == 1:
        return user_df
    else:
        return user_df[:-1]
from tqdm import tqdm
tqdm.pandas()

# item_hist = item.groupby('user_id').progress_apply(hist_func).reset_index(drop=True)

In [18]:
# item_hist.to_csv("item_hist.csv",index=False)
item_hist = pd.read_csv("item_hist.csv",index_col=None)

In [19]:
item_hist.shape, item_last.shape, item.shape

((5525076, 4), (913707, 4), (6319946, 4))

In [29]:
check = item_last.merge(item_hist,on=["user_id", "item_id"])
check.shape, check[check.latest_timestamp_x == check.latest_timestamp_y].head()

((118837, 6),
    user_id  item_id  latest_timestamp_x  frequency_x  latest_timestamp_y  \
 0       19  I279860          1587772800            1          1587772800   
 1       26  I339856          1581379200            1          1581379200   
 2       47  I180868          1581292800            1          1581292800   
 3       58  I348207          1584316800            1          1584316800   
 4       76  I210903          1586736000            1          1586736000   
 
    frequency_y  
 0            1  
 1            1  
 2            1  
 3            1  
 4            1  )

In [ ]:
item.set_index(["user_id","item_id"]).loc[51].loc["I145321"]

latest_timestamp  frequency
user_id item_id                             
51      I280244        1581724800          1
        I89484         1583280000          1
        I80529         1584921600          1
        I76294         1584921600          1
        I72579         1584921600          2
        I78233         1584921600          2
        I145321        1584921600          2
        I333846        1584921600          2

In [ ]:
item_last.latest_timestamp.mean(), item_hist.latest_timestamp.mean()

(1585465311.0250878, 1584377814.3402915)

In [ ]:
item_hist.item_id.nunique(),item.item_id.nunique()

(312038, 328823)

In [ ]:
item.latest_timestamp.max() - item.latest_timestamp.min()
item.latest_timestamp.max() ,item.latest_timestamp.min()

(1588118400, 1580428800)

In [ ]:
user_feature.shape, item.shape,user_target.shape

((1342378, 137), (6319946, 4), (250343, 1))

In [ ]:
popular_items = item_hist.groupby("item_id")["latest_timestamp"].count()\
    .sort_values(ascending=False)
popular_2 = item_hist["item_id"].value_counts()[:]
popular_items[:10],popular_2[:10]

(item_id
 I102695    20798
 I103751    19440
 I302808    19390
 I177269    18896
 I31876     15219
 I46072     14590
 I337068    14415
 I193280    12601
 I196644    11955
 I7453      11882
 Name: latest_timestamp, dtype: int64,
 item_id
 I102695    20798
 I103751    19440
 I302808    19390
 I177269    18896
 I31876     15219
 I46072     14590
 I337068    14415
 I193280    12601
 I196644    11955
 I7453      11882
 Name: count, dtype: int64)

In [ ]:
item_hist.user_id.max()

### train valid split

In [20]:
from sklearn.model_selection import GroupKFold
kFold = GroupKFold(n_splits=5)

for i, (trn_idx, val_idx) in enumerate(kFold.split(item_hist,item_hist,groups=item_hist["user_id"])):
    item_trn = item_hist.iloc[trn_idx]
    item_val = item_hist.iloc[val_idx]
    break


In [74]:
item_trn = item_hist.iloc[trn_idx]
item_val = item_hist.iloc[val_idx]

In [110]:
item_trn.user_id.nunique(),item_val.user_id.nunique(),\
  item_trn.item_id.nunique(),item_val.user_id.nunique()

(730966, 182741, 283142, 182741)

### make dataset

In [85]:

N = 150000
# top_items = list(item["item_id"].value_counts().index[:N].values)
top_items = [item2idx[x] for x in item.item_id.values]
# N_U = 450000
# top_users = list(item["user_id"].value_counts().index[:N_U].values)

train_data_base = item[["user_id","item_id"]]
train_data_base.loc[:,"item_id"] = train_data_base.item_id.apply(lambda x:item2idx[x])

# train_data_base = train_data_base[train_data_base.item_id.isin(top_items)]
# train_data_base = train_data_base[train_data_base.user_id.isin(top_users)]

# print("choosing top", N,N_U, item.shape[0],train_data_base.shape[0])

In [122]:
import random
import numpy as np

train_list = []
for neg_num in range(10):
    print(neg_num, "...")
    train_data_base.loc[:,"neg_item_id"] = np.random.choice(top_items,train_data_base.shape[0])
    # train_data_base.loc[:,"neg_item_id"] = train_data_base.neg_item_id.apply(lambda x:item2idx[x])
    train_list.append(train_data_base.copy())
train_data = pd.concat(train_list)
train_data.head(5)

0 ...
1 ...
2 ...
3 ...
4 ...
5 ...
6 ...
7 ...
8 ...
9 ...


,user_id,item_id,neg_item_id
1669837,2,2719,229
1669838,2,19194,10138
4139901,3,17221,11946
4139902,3,3770,10555
4139903,3,17225,1135


In [87]:
train_data.shape, train_data_base.shape

((63199460, 3), (6319946, 3))

In [123]:
print(train_data.shape, train_data.user_id.nunique(),train_data.item_id.nunique())
print(train_data.neg_item_id.nunique(), train_data_base.shape)
print(train_data[["user_id","item_id"]].duplicated().sum(),\
            train_data[["user_id","item_id","neg_item_id"]].duplicated().sum())

(44200600, 119) 730966 283142
328823 (4420060, 3)
39780540 592


In [ ]:
train_data_base.item_id.value_counts().describe()

count    30000.000000
mean        91.420233
std        281.732354
min          2.000000
25%         15.000000
50%         25.000000
75%         57.000000
max       8603.000000
Name: count, dtype: float64

In [ ]:
train_data_base.user_id.value_counts().describe()

count    199754.000000
mean         13.729923
std          13.007129
min           1.000000
25%           9.000000
50%          11.000000
75%          15.000000
max        1055.000000
Name: count, dtype: float64

In [349]:
item.item_id.value_counts().describe()

count    328823.000000
mean         19.219903
std         196.719348
min           1.000000
25%           1.000000
50%           2.000000
75%           6.000000
max       23663.000000
Name: count, dtype: float64

In [ ]:
train_data.head()

,user_id,item_id,neg_item_id
0,484,0,108872
1,484,1,13764
2,815,2,108122
3,998,3,121519
4,998,4,261507


In [ ]:
train_data.item_id.value_counts().sort_values(ascending=False)[149999:]

item_id
44052    8
Name: count, dtype: int64

In [ ]:
train_data.user_id.nunique(),item_val.user_id.nunique()

(908075, 182741)

In [ ]:
# from sklearn.utils import shuffle
# train_data = shuffle(train_data)
# train_data


In [ ]:
train_data.iloc[:10].sample(5)

,user_id,item_id,neg_item_id
5,998,5,147028
6,998,6,7095
4,998,4,171863
0,484,0,44927
2,815,2,8695


In [ ]:
user_target.user_id.max()

1640953

In [36]:
list(user_feature.columns[16:])

['A1',
 'A2',
 'A3',
 'B1',
 'B10',
 'B11',
 'B12',
 'B13',
 'B14',
 'B15',
 'B16',
 'B17',
 'B18',
 'B19',
 'B2',
 'B20',
 'B21',
 'B22',
 'B23',
 'B24',
 'B25',
 'B26',
 'B27',
 'B28',
 'B29',
 'B3',
 'B30',
 'B31',
 'B32',
 'B33',
 'B34',
 'B35',
 'B36',
 'B37',
 'B38',
 'B39',
 'B4',
 'B40',
 'B41',
 'B42',
 'B43',
 'B44',
 'B45',
 'B46',
 'B47',
 'B48',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'C1',
 'C10',
 'C11',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'D1',
 'D2',
 'E1',
 'E10',
 'E11',
 'E12',
 'E2',
 'E3',
 'E4',
 'E5',
 'E6',
 'E7',
 'E8',
 'E9',
 'F1',
 'F10',
 'F2',
 'F3',
 'F4',
 'F5',
 'F6',
 'F7',
 'F8',
 'F9',
 'feature_list1_0',
 'feature_list1_1',
 'feature_list1_2',
 'feature_list1_3',
 'feature_list1_4',
 'feature_list1_5',
 'feature_list1_6',
 'feature_list1_7',
 'feature_list1_8',
 'feature_list1_9',
 'feature_list3_0',
 'feature_list3_1',
 'feature_list3_2',
 'feature_list3_3',
 'feature_list3_4',
 'feature_list3_5',
 'feature_list3_6',
 'feature_lis

In [24]:
user_feature = user_feature.set_index("user_id")

fea_cols = list(user_feature.columns[15:])
num_fea = len(fea_cols)

# fea_cols = list(user_feature.columns[15:])
user_feature[fea_cols].head()

,A1,A2,A3,B1,B10,B11,B12,B13,B14,B15,...,feature_list5_0,feature_list5_1,feature_list5_2,feature_list5_3,feature_list5_4,feature_list5_5,feature_list5_6,feature_list5_7,feature_list5_8,feature_list5_9
user_id,,,,,,,,,,,,,,,,,,,,,
484,True,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,False,True,True,False,True
815,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
998,True,False,False,True,False,False,False,False,False,False,...,True,True,True,True,True,False,True,False,False,True
1213,True,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1462,True,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False


In [72]:
fea_cols

['A1',
 'A2',
 'A3',
 'B1',
 'B10',
 'B11',
 'B12',
 'B13',
 'B14',
 'B15',
 'B16',
 'B17',
 'B18',
 'B19',
 'B2',
 'B20',
 'B21',
 'B22',
 'B23',
 'B24',
 'B25',
 'B26',
 'B27',
 'B28',
 'B29',
 'B3',
 'B30',
 'B31',
 'B32',
 'B33',
 'B34',
 'B35',
 'B36',
 'B37',
 'B38',
 'B39',
 'B4',
 'B40',
 'B41',
 'B42',
 'B43',
 'B44',
 'B45',
 'B46',
 'B47',
 'B48',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'C1',
 'C10',
 'C11',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'D1',
 'D2',
 'E1',
 'E10',
 'E11',
 'E12',
 'E2',
 'E3',
 'E4',
 'E5',
 'E6',
 'E7',
 'E8',
 'E9',
 'F1',
 'F10',
 'F2',
 'F3',
 'F4',
 'F5',
 'F6',
 'F7',
 'F8',
 'F9',
 'feature_list1_0',
 'feature_list1_1',
 'feature_list1_2',
 'feature_list1_3',
 'feature_list1_4',
 'feature_list1_5',
 'feature_list1_6',
 'feature_list1_7',
 'feature_list1_8',
 'feature_list1_9',
 'feature_list3_0',
 'feature_list3_1',
 'feature_list3_2',
 'feature_list3_3',
 'feature_list3_4',
 'feature_list3_5',
 'feature_list3_6',
 'feature_lis

In [100]:
train_data = train_data.merge(user_feature[fea_cols],on="user_id")


In [81]:
train_data_sampled = train_data[train_data.columns[:]].sample(frac=1)#5000000)
train_data_sampled.shape, train_data_sampled.head()

((44200600, 3),
          user_id item_id  neg_item_id
 2380881   703693    4451         5296
 4127183  1225106  117319       165002
 4370639  1296981    1686          640
 4939486  1467123    2110         6483
 1411792   417857  124123         1760)

In [75]:
user_feature[fea_cols].memory_usage().sum()/1000000 * 40

5532.88832

In [72]:
user_feature[fea_cols].shape

(940785, 115)

In [71]:
train_data.memory_usage().sum() / 1000000

1237.6168

In [57]:
user_feature[fea_cols].loc[3].values.shape

(116,)

In [70]:
train_dataset[0]

(607107,
 array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       dtype=float32),
 26360,
 56755)

In [108]:
train_data_sampled.head()

,user_id,item_id,neg_item_id
2380881,703693,4451,5296
4127183,1225106,117319,165002
4370639,1296981,1686,640
4939486,1467123,2110,6483
1411792,417857,124123,1760


In [123]:
import importlib
import BPR_lib
importlib.reload(BPR_lib)
from BPR_lib import BPRData2
import torch.utils.data as data
# train_dataset = BPRData2(train_data_sampled[train_data_sampled.columns[:3]].values,\
                        #   user_feature[fea_cols[:].copy()],True)
train_dataset = BPRData2(train_data.values, user_feature[fea_cols],True)
train_loader = data.DataLoader(train_dataset,batch_size=1024,shuffle=True,num_workers=0)



In [95]:
train_data.shape

(63199460, 3)

In [108]:
len(train_dataset),train_dataset[0],

(63199460,
 (2,
  array([0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1.],
        dtype=float32),
  2719,
  2323))

In [ ]:
for user,user_fea,item_i,item_j in tqdm(train_loader):
    item_i.cuda()
    # print(user_fea.shape,item_i)
    # print(item_i.dtype)

In [ ]:
train_data.iloc[0].values.astype(np.float32)[3:].shape

(0,)

In [ ]:
user_feature.shape[1],train_dataset[0][0].shape

(112, ())

In [ ]:
np.sum(train_data.iloc[0].values[3:].astype(np.float32)!=train_dataset[0][0])

0

In [135]:
train_data.neg_item_id.nunique(),train_data.item_id.nunique(),\
    train_data.user_id.nunique(), train_data.user_id.max(),

(328819, 328823, 913707, 1640955)

### Training

In [103]:
import importlib
import BPR_lib
importlib.reload(BPR_lib)
from BPR_lib import BPR,BPR2
import torch.optim as optim
import torch
factor_num = 64
# model1 = BPR(user_num, click_num, factor_num)

model = BPR2(item.user_id.max()+1, num_fea, len(item_ids), factor_num1=32,factor_num2=32)
# model = BPR2(num_fea,len(item_ids),factor_num)
args_lambda = 0.001
args_lr = 0.01
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=args_lr, weight_decay=args_lambda)

In [ ]:
train_data.user_id.max(),item.user_id.max(),user_target.user_id.max()

(1640954, 1640955, 1640953)

In [50]:

pre_i,pre_j = model(torch.tensor(train_dataset[0][0]).cuda(),\
                    torch.tensor(train_dataset[0][1]).cuda(),\
      torch.tensor(train_dataset[0][2]).cuda(),torch.tensor(train_dataset[0][3]).cuda())
pre_i,pre_j

(tensor(0.0818, device='cuda:0', grad_fn=<SumBackward1>),
 tensor(0.0058, device='cuda:0', grad_fn=<SumBackward1>))

In [43]:
torch.tensor(train_dataset[0][1])

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [ ]:
len(train_data),len(train_loader)

(27426070, 26784)

In [ ]:
torch.tensor([-2,-3,-4]).sigmoid().log().sum(),torch.tensor([2,3,4]).sigmoid().log().sum()

tensor(-9.1937)

In [124]:
from tqdm import tqdm

ii = 0
for epoch in range(1):
    model.train()
    model.cuda()
    for user, user_fea,item_i,item_j in tqdm(train_loader):
        user_fea = user_fea.cuda()
        user = user.cuda()
        item_i = item_i.cuda()
        item_j = item_j.cuda()
        # print(user_fea.shape,item_i)
        model.zero_grad()
        pred_i,pred_j = model(user, user_fea,item_i,item_j)
        ii+=1
        if ii % 1000 == 0:
            print("check", [list[x.detach().cpu().numpy()] for x in [pred_i[:5],pred_j[:5],\
            model.user_embed(user,user_fea)[0][:10],model.item_embed(item_i)[0][:10]]])
        loss = - (pred_i - pred_j).sigmoid().log().sum()
        loss.backward()
        optimizer.step()
    # print(u_embed.shape,embed_i.shape)

 96%|█████████▌| 59001/61719 [1:12:28<03:13, 14.07it/s]

check [list[array([3.1488018, 1.3833923, 0.2210561, 0.9818157, 0.9205785],
      dtype=float32)], list[array([-0.4082346 ,  1.0236664 , -1.9304743 , -0.35549483,  0.28632563],
      dtype=float32)], list[array([-2.1843472 , -0.4381886 , -0.05299413, -0.52643865, -0.120143  ,
        0.07249117, -1.4373157 ,  1.5713549 ,  0.769473  ,  1.7390727 ],
      dtype=float32)], list[array([-0.1341399 , -0.05452934,  0.07267233, -0.04685592,  0.05998017,
       -0.11189521,  0.18361548,  0.18742761, -0.29663113,  0.3096957 ],
      dtype=float32)]]


 97%|█████████▋| 60001/61719 [1:13:43<02:11, 13.10it/s]

check [list[array([ 1.544491  ,  0.9427264 , -0.23047781,  1.449806  ,  1.5843006 ],
      dtype=float32)], list[array([-0.63277185,  0.543048  ,  0.50256616,  0.81974757,  0.8716283 ],
      dtype=float32)], list[array([-1.6110687 , -1.1634713 , -0.30551505, -1.2000492 ,  0.65162915,
       -1.3268124 , -1.5972018 , -1.6157709 ,  0.96970063, -0.21032652],
      dtype=float32)], list[array([ 0.05915433, -0.08363575, -0.02403608, -0.08300272, -0.01856421,
       -0.32198402, -0.15081598, -0.28678733,  0.16061409, -0.19186157],
      dtype=float32)]]


 99%|█████████▉| 61001/61719 [1:14:59<00:53, 13.38it/s]

check [list[array([1.2198668 , 0.77299345, 0.56073433, 2.6902957 , 0.70911306],
      dtype=float32)], list[array([ 0.01570423, -0.7263076 ,  0.71389335,  0.3934567 ,  1.8236034 ],
      dtype=float32)], list[array([ 0.20961246, -1.5382155 ,  0.27710503, -0.5951878 ,  0.5246996 ,
       -0.66802025, -0.12668094, -0.7564934 ,  1.9947782 , -0.51241016],
      dtype=float32)], list[array([-0.03364911,  0.03187267,  0.03051153, -0.2397758 ,  0.16054124,
        0.03392074,  0.04843102, -0.33200192, -0.01753592, -0.06274085],
      dtype=float32)]]


100%|██████████| 61719/61719 [1:16:00<00:00, 13.53it/s]


In [64]:
train_data

,user_id,item_id,neg_item_id,A1,A2,A3,B1,B10,B11,B12,...,feature_list5_0,feature_list5_1,feature_list5_2,feature_list5_3,feature_list5_4,feature_list5_5,feature_list5_6,feature_list5_7,feature_list5_8,feature_list5_9
0,3,17221,283141,True,False,False,True,False,False,False,...,True,False,False,False,True,False,False,False,True,True
1,3,3770,214316,True,False,False,True,False,False,False,...,True,False,False,False,True,False,False,False,True,True
2,3,17225,221603,True,False,False,True,False,False,False,...,True,False,False,False,True,False,False,False,True,True
3,3,16696,145221,True,False,False,True,False,False,False,...,True,False,False,False,True,False,False,False,True,True
4,5,22165,119061,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44200595,1640944,10961,142828,True,False,False,False,False,False,False,...,True,True,True,True,True,False,True,True,True,True
44200596,1640944,2110,100493,True,False,False,False,False,False,False,...,True,True,True,True,True,False,True,True,True,True
44200597,1640949,2395,89824,True,False,False,False,False,False,False,...,True,True,True,True,True,False,True,True,True,True
44200598,1640955,46358,162890,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [141]:
# torch.save(model.state_dict(),"BPR_20w_3w_sample10_factor64.pth")
model.load_state_dict(torch.load("BPR_sample10_factor32_32_userfea_ep2.pth"))

<All keys matched successfully>

In [ ]:
user_fea.shape

torch.Size([1024])

In [125]:
torch.save(model.state_dict(),"BPR_sample10_factor32_32_userfea_ep22.pth")

In [ ]:
np.concatenate([[[1,2],[3,4]],[[5,6]]],axis=1)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 2 and the array at index 1 has size 1

In [ ]:
idx2item[int(item_i[0].numpy())]

'I32641'

In [126]:
start = 3000000

for ii in range(start, start+20):
    uu,u_fea,item_i,item_j = train_dataset[ii]
    uu = torch.tensor(uu).cuda()
    ufea = torch.tensor(u_fea).cuda()
    item_fea_i = torch.tensor(item_i).cuda()
    item_fea_j = torch.tensor(item_j).cuda()
    pred_i2,pred_j2 = model(uu,ufea,item_fea_i,item_fea_j)
    print([float(x.detach().cpu().numpy()) for x in [pred_i2,pred_j2]])

[1.8601807355880737, -1.3167762756347656]
[1.0709214210510254, 0.15930402278900146]
[0.8781939744949341, -2.740041732788086]
[1.975198745727539, 1.0187382698059082]
[3.7787208557128906, -0.2407495379447937]
[2.377258062362671, 2.643624782562256]
[0.48873597383499146, -2.371997356414795]
[-0.29836025834083557, -2.456918239593506]
[0.7253111004829407, 1.200806736946106]
[0.6671362519264221, 0.8465331792831421]
[1.6458489894866943, -0.04380291700363159]
[1.2793346643447876, -1.1004817485809326]
[0.8349499702453613, 0.7340946197509766]
[-0.48531392216682434, -0.15152427554130554]
[1.115967035293579, -0.48731738328933716]
[1.8160433769226074, 0.9013816714286804]
[0.5302735567092896, 1.3197439908981323]
[2.2373013496398926, -1.5094951391220093]
[1.6348556280136108, 2.478978395462036]
[1.5769932270050049, 4.246208667755127]


In [75]:
len(trn_item_embeds), trn_item_embeds[0].shape

(328823, (64,))

In [142]:
trn_item_embeds = []
all_items = [item2idx[x] for x in item_ids]
for ii in tqdm(range(0,len(all_items),64)):
    # item_id_idx = [item2idx[x] for x in top_items[ii:ii+64]]
    item_embed = model.item_embed(torch.tensor(all_items[ii:ii+64],dtype=torch.int32).cuda())
    trn_item_embeds.extend(item_embed.detach().cpu().numpy())
import faiss
index = faiss.IndexFlatIP(factor_num)
index.add(np.array(trn_item_embeds))
len(trn_item_embeds)


100%|██████████| 5138/5138 [00:02<00:00, 2382.02it/s]


328823

In [143]:
# test_user_ids = list(item_trn.user_id.unique())
# gt_item = item_trn.groupby("user_id")["item_id"].apply(list)
# gt_item2 = item_last[item_last.user_id.isin(test_user_ids)].set_index("user_id")
# np.random.shuffle(test_user_ids)
user_feature_t = user_feature[fea_cols]
count1 =0 
count2 = 0
total1 = 0
total2 = 0
check = 0
with tqdm(total=len(test_user_ids)) as pbar:
    for test_user in test_user_ids:
        user_embed = model.user_embed(torch.tensor(test_user).cuda(),\
            torch.tensor(user_feature_t.loc[test_user].values.astype(np.float32)).cuda()).detach().cpu().numpy()
        # user_embed[:] = 0
        # sims2 = np.dot(user_embed,np.array(trn_item_embeds).T)
        # arts_fetch2 = np.random.choice(trn_item_ids,30000)
        # arts_fetch1 = np.array(top_items)[np.argsort(sims2)[-1500:]]
        # arts_fetch2 = np.array(top_items)[np.argsort(sims2)[:148500]]
        sim,idx =  index.search(np.array([user_embed]),3000)

        gt_arts = gt_item.loc[test_user]
        gt_arts = [item2idx[x] for x in gt_arts]
        gt_arts2 = gt_item2.loc[test_user].item_id
        gt_arts2 = [item2idx[x] for x in [gt_arts2]]
        count1+=len(set(gt_arts).intersection(set(idx[0][:])))
        count2+=len(set(gt_arts2).intersection(set(idx[0][:])))
        check+=len(set(gt_arts).intersection(set(gt_arts2)))
        total1+=len(gt_arts)
        total2+=1#len(gt_arts2)
        pbar.update(1)
        pbar.set_postfix(h1=count1,t=total1,h2=count2,t2=total2,c=check)

  0%|          | 330/730966 [00:04<2:46:28, 73.14it/s, c=53, h1=619, h2=91, t=1977, t2=330]


KeyboardInterrupt: 

In [204]:
# test_user_ids = list(item_trn.user_id.unique())
# gt_item = item_trn.groupby("user_id")["item_id"].apply(list)
# gt_item2 = item_last[item_last.user_id.isin(test_user_ids)].set_index("user_id")
# np.random.shuffle(test_user_ids)
user_feature_t = user_feature[fea_cols]
count1 =0 
count2 = 0
total1 = 0
total2 = 0
check = 0
with tqdm(total=len(test_user_ids)) as pbar:
    for test_user in test_user_ids:
        user_embed = model.user_embed(torch.tensor(test_user).cuda(),\
            torch.tensor(user_feature_t.loc[test_user].values.astype(np.float32)).cuda()).detach().cpu().numpy()
        # user_embed[:] = 0
        # sims2 = np.dot(user_embed,np.array(trn_item_embeds).T)
        # arts_fetch2 = np.random.choice(trn_item_ids,30000)
        # arts_fetch1 = np.array(top_items)[np.argsort(sims2)[-1500:]]
        # arts_fetch2 = np.array(top_items)[np.argsort(sims2)[:148500]]
        sim,idx =  index.search(np.array([user_embed]),3000)

        gt_arts = gt_item.loc[test_user]
        gt_arts = [item2idx[x] for x in gt_arts]
        gt_arts2 = gt_item2.loc[test_user].item_id
        gt_arts2 = [item2idx[x] for x in [gt_arts2]]
        count1+=len(set(gt_arts).intersection(set(idx[0][:])))
        count2+=len(set(gt_arts2).intersection(set(idx[0][:])))
        check+=len(set(gt_arts).intersection(set(gt_arts2)))
        total1+=len(gt_arts)
        total2+=1#len(gt_arts2)
        pbar.update(1)
        pbar.set_postfix(h1=count1,t=total1,h2=count2,t2=total2,c=check)

  0%|          | 653/730966 [00:10<3:18:23, 61.35it/s, c=90, h1=2434, h2=412, t=3910, t2=653]


KeyboardInterrupt: 

In [135]:
len(gt_item.index.unique())

730966

In [120]:
set(test_user_ids).intersection(train_data.user_id.values)

set()

In [134]:
# gt_item_val = item_val.groupby("user_id")["item_id"].apply(list)
# gt_item_val2 = item_last[item_last.user_id.isin(test_user_ids_val)].set_index("user_id")
# np.random.shuffle(test_user_ids_val)
# user_feature_t = user_feature[fea_cols]
count1 =0 
count2 = 0
total1 = 0
total2 = 0
check = 0
with tqdm(total=len(test_user_ids_val)) as pbar:
    for test_user in test_user_ids_val:
        user_embed = model.user_embed(torch.tensor(test_user).cuda(),\
            torch.tensor(user_feature_t.loc[test_user].values.astype(np.float32)).cuda()).detach().cpu().numpy()

        user_embed[32:] = 0
        # sims2 = np.dot(user_embed,np.array(trn_item_embeds).T)
        # arts_fetch2 = np.random.choice(trn_item_ids,30000)
        # arts_fetch1 = np.array(top_items)[np.argsort(sims2)[-1500:]]
        # arts_fetch2 = np.array(top_items)[np.argsort(sims2)[:148500]]
        sim,idx =  index.search(np.array([user_embed]),3000)
        
        gt_arts = gt_item_val.loc[test_user]
        gt_arts = [item2idx[x] for x in gt_arts]
        gt_arts2 = gt_item_val2.loc[test_user].item_id
        gt_arts2 = [item2idx[x] for x in [gt_arts2]]
        count1+=len(set(gt_arts).intersection(set(idx[0])))
        count2+=len(set(gt_arts2).intersection(set(idx[0])))
        check+=len(set(gt_arts).intersection(set(gt_arts2)))
        total1+=len(gt_arts)
        total2+=1#len(gt_arts2)
        pbar.update(1)
        pbar.set_postfix(h1=count1,t=total1,h2=count2,t2=total2,c=check)

  0%|          | 772/182741 [00:10<41:26, 73.18it/s, c=100, h1=1387, h2=214, t=5068, t2=772]


KeyboardInterrupt: 

In [69]:
# test_user_ids_val = list(item_val.user_id.unique())
# gt_item_val = item_val.groupby("user_id")["item_id"].apply(list)
# gt_item_val2 = item_last[item_last.user_id.isin(test_user_ids_val)].set_index("user_id")
# np.random.shuffle(test_user_ids_val)
user_feature_t = user_feature[fea_cols]
count1 =0 
count2 = 0
total1 = 0
total2 = 0
check = 0
with tqdm(total=len(test_user_ids_val)) as pbar:
    for test_user in test_user_ids_val:
        user_embed = model.user_embed(torch.tensor(test_user).cuda(),\
            torch.tensor(user_feature_t.loc[test_user].values.astype(np.float32)).cuda()).detach().cpu().numpy()
        # user_embed = model.user_embed(torch.tensor(1300000).cuda(),\
        #     torch.tensor(np.ones([num_fea]).astype(np.float32)).cuda()).detach().cpu().numpy()
        # user_embed[32:]=0
        # sims2 = np.dot(user_embed,np.array(trn_item_embeds).T)
        # arts_fetch2 = np.random.choice(trn_item_ids,30000)
        # arts_fetch1 = np.array(top_items)[np.argsort(sims2)[-1500:]]
        # arts_fetch2 = np.array(top_items)[np.argsort(sims2)[:148500]]
        sim,idx =  index.search(np.array([user_embed]),75)
        
        gt_arts = gt_item_val.loc[test_user]
        gt_arts = [item2idx[x] for x in gt_arts]
        gt_arts2 = gt_item_val2.loc[test_user].item_id
        gt_arts2 = [item2idx[x] for x in [gt_arts2]]
        count1+=len(set(gt_arts).intersection(set(idx[0])))
        count2+=len(set(gt_arts2).intersection(set(idx[0])))
        check+=len(set(gt_arts).intersection(set(gt_arts2)))
        total1+=len(gt_arts)
        total2+=1#len(gt_arts2)
        pbar.update(1)
        pbar.set_postfix(h1=count1,t=total1,h2=count2,t2=total2,c=check)

  0%|          | 819/182741 [00:10<39:10, 77.39it/s, c=105, h1=494, h2=112, t=5349, t2=819]


KeyboardInterrupt: 

In [70]:
user_embed

array([-0.88718164,  2.5703428 , -5.1009116 , -0.22926289, -1.3572013 ,
       -5.089199  ,  2.1503918 , -3.2485995 , -1.8210777 ,  2.3471398 ,
       -0.3205343 ,  1.872113  ,  4.842472  , -1.8419566 ,  0.08358112,
        2.7726512 ,  0.55332446, -1.6370053 , -5.521165  , -5.712897  ,
       -5.5914035 ,  3.4464967 , -1.0202622 , -6.0686846 ,  0.44085923,
        2.9671547 ,  0.76929575,  0.72559875,  1.8588591 ,  4.3710546 ,
       -1.189645  ,  0.1610449 ,  1.475866  ,  1.2095798 , -0.21995635,
        0.9593642 , -0.515788  ,  0.480631  ,  0.6215424 , -0.99999833,
       -0.7969965 ,  0.22732386,  0.38497117, -0.19358094, -0.83543867,
       -0.1885616 , -0.13287397, -0.351429  ,  1.2515988 , -0.05039941,
       -0.4200672 ,  1.1098335 ,  0.68658584, -0.8058521 , -0.8579219 ,
        0.03770611, -1.0487317 , -0.48281243, -0.41328782, -0.38665026,
       -1.1660242 ,  0.55318165, -0.6177625 ,  0.49004832], dtype=float32)

In [63]:
item.user_id.value_counts().loc[657726]

1784

In [43]:
user_embed.shape

(64,)

In [162]:
user_embed

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [138]:
len(gt_item_val.index.unique())

182741

In [100]:
set(gt_arts2)

{'0', '1', '4', '5', '8', 'I'}

In [287]:
trn_item_embeds[25300]

array([-0.00205872,  0.01067198,  0.00442559,  0.00387845, -0.00243466,
       -0.00018402,  0.00366135, -0.00064728,  0.00664593, -0.00826498,
        0.00831544,  0.00995937,  0.00281455, -0.00441395,  0.00028359,
       -0.00266338, -0.00372765, -0.00231449, -0.00027135, -0.00983913,
        0.00378682, -0.00506463,  0.00596974,  0.00934513, -0.00212543,
       -0.00384673,  0.00536456,  0.00757025, -0.0020385 , -0.00775156,
        0.00721003,  0.00400864,  0.00114823, -0.00374699, -0.0127686 ,
       -0.00113352, -0.00088708,  0.0096148 ,  0.01219288,  0.00373809,
       -0.00394224,  0.00305708, -0.00365596, -0.00378627, -0.00196633,
        0.00321073,  0.01195292, -0.00072244, -0.00443617, -0.00162997,
       -0.00321434, -0.01534081,  0.01623973,  0.00530946,  0.00361922,
        0.00076687, -0.00397851,  0.0058771 ,  0.00251416, -0.00709711,
        0.01413247,  0.00443728, -0.00552901, -0.00657055], dtype=float32)

In [266]:
max(np.array(top_items)[np.argsort(sims2)[-1500:]])

25300

In [260]:
np.array(top_items)[np.argsort(sims2)[-1500:]]+1

array([ 4301, 12498,  3350, ...,    86,   862,   357])

In [ ]:
len(trn_item_embeds)

50000

In [264]:
len(sims2)

150000

In [ ]:
len(item_embed_val),len(item_embed_val[0])

(328823, 100)

In [ ]:
item_hist_val_idx = item_hist_val.groupby("user_id")["item_id"].apply(list)

In [ ]:
val_user_id = item_hist_val.user_id.unique()
len(val_user_id)


182741

In [ ]:
user_feature_val = user_feature.loc[val_user_id]
user_feature_val.shape

(182741, 986)

In [ ]:
train_data.item_id.nunique()

50000

In [ ]:
user_embed_val = []


for ii in tqdm(range(0,user_feature_val.shape[0],64)):
    user_feature_t = user_feature_val.iloc[ii:ii+64].values
    # user_feature_t = np.zeros([128,986])
    user_feature_t = user_feature_t.astype(np.float32)
    # print(user_feature_t.shape)
    user_embed = model.user_embed(torch.tensor(user_feature_t).cuda())
    user_embed = user_embed.detach().cpu().numpy()
#     print(user_embed.shape)
    user_embed_val.extend(user_embed)


100%|██████████| 2856/2856 [00:17<00:00, 162.46it/s]


In [ ]:
len(user_embed_val), user_embed_val[182740].shape,user_embed_val[0]

(182741,
 (100,),
 array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32))

In [ ]:
import faiss
index = faiss.IndexFlatIP(factor_num)
index.add(item_embed_val)

NameError: name 'factor_num' is not defined